In [1]:
import random
import psycopg2
from datetime import datetime as DT
from datetime import timedelta

In [2]:
def get_random_date(start, end):
    delta = end - start
    return start + timedelta(random.randint(0, delta.days))

In [3]:

def get_user_query(i):
    NAME_M = ['Антон', 'Андрей', 'Иван', 'Григорий', 'Александр', 'Никита', 'Руслан', 'Кирилл', 'Ибрагим', 'Анатолий', 'Михаил', 'Денис']
    NAME_F = ['Анастасия', 'Татьяна', 'Надежда', 'Марина', 'Екатерина', 'Ирина', 'Анна', 'Мария', 'Дарья']
    SURNAME = ['Иванов','Петров','Кочетков','Гончаров','Печкин','Абрамов','Львов','Задорнов','Измайлов','Баранов','Комаров','Лыткарин','Морев','Колчанов','Смирнов','Харитонов','Ворошилов']
    FATHERS = ['Ивано', 'Петро', 'Антоно', 'Андрее','Григорье','Александро','Руслано','Ибрагимо','Анатолье','Михайло', 'Денисо']
    LOGIN = ['cat', 'fox', 'owl', 'bear', 'parrot', 'pen', 'puma', 'tiger', 'lion', 'ostrich', 'sheep', 'dog', 'cow', 'shrek', 'shadow']
    EMAIL = ['@gmail.com', '@yandex.ru', '@mail.ru', '@mephi.ru']
    
    query = 'INSERT INTO "user" (full_name, birth_date, gender, email, phone, verification, work_experience) VALUES'
    
    male_flag = random.randint(0, 1)
    
    if (male_flag == 0):
        name = random.choice(NAME_M)+' '+random.choice(FATHERS)+'вич'+' '+random.choice(SURNAME)
        gender = 'male'
    else:
        name = random.choice(NAME_F)+' '+random.choice(FATHERS)+'вна'+' '+random.choice(SURNAME)+'а'
        gender = 'female'

    start_dt = DT.strptime('01.01.1950', '%d.%m.%Y')
    end_dt = DT.strptime('01.01.2004', '%d.%m.%Y')

    birth_date = get_random_date(start_dt, end_dt)
    email = random.choice(LOGIN)+str(i)+random.choice(EMAIL)
    phone_number = '89'+random.choice(['25','78','80','09','04'])+str(random.randint(1000000, 9999999))
    
    if (random.randint(0,1) == 0):
        documents_checked = 'false'
    else:
        documents_checked = 'true'
    
    max_experience = (DT.strptime('05.03.2023', '%d.%m.%Y') - birth_date).total_seconds() / 3600 / 24 / 365 - 18
    work_experience = random.randint(1, int(max_experience))
    
    query += f'(\'{name}\', \'{birth_date}\', \'{gender}\', \'{email}\', \'{phone_number}\', \'{documents_checked}\', {work_experience})'
    query += ' RETURNING user_id'
    
    return query


def add100users():
    conn = psycopg2.connect(user='postgres', password='1234', host='localhost', port='5432', database="youdo")
    cur = conn.cursor()
    
    for i in range(100):
        user_query = get_user_query(i)
        cur.execute(user_query)
    
    conn.commit()
    cur.close()
    conn.close()

In [4]:
add100users()

In [7]:
# fill albums

import random
import psycopg2

user_id_query = 'SELECT user_id FROM "user"'

conn = psycopg2.connect(user='postgres', password='1234', host='localhost', port='5432', database="youdo")
cur = conn.cursor()

cur.execute(user_id_query)
res = cur.fetchall()
ids = []
for i in res:
    ids.append(i[0])

def get_album_query(id):
    NAME_1 = ["Good", "Hard", "Excellent", "Tiny", "Cute", "Grand", 
              "Ideal", "Default", "Important", "Quick"]
    NAME_2 = ["work", "task", "experience", "job", "diagnosis",
              "deal"]
    name = random.choice(NAME_1) + " " + random.choice(NAME_2)
    des = "This was really " + random.choice(NAME_1).lower() + " " + random.choice(NAME_2) + " that took " + str(random.randint(2, 10)) + " " + random.choice(["hours", "days", "months"]) + "."
    query = "INSERT INTO album(\"name\", description, owner_id) VALUES "
    query += f'(\'{name}\', \'{des}\', \'{id}\')'
    return query

for i in range(100):
    id = random.choice(ids)
    query = get_album_query(id)
    cur.execute(query)
    
conn.commit()
cur.close()
conn.close()


In [4]:
# fill account

import random
import psycopg2

conn = psycopg2.connect(user='postgres', password='1234', host='localhost', port='5432', database="youdo")
cur = conn.cursor()

user_id_query = 'SELECT user_id FROM "user"'

cur.execute(user_id_query)
res = cur.fetchall()
ids = []
for i in res:
    ids.append(i[0])

# "пополнение счета"
for i in range(100):
    id = random.choice(ids)
    money = random.randint(40,100)*10
    cur.execute(f"SELECT account::numeric::int FROM \"user\" WHERE user_id={id}")
    before = cur.fetchall()[0][0]
    account_query = f"UPDATE \"user\" SET "\
    f"account={int(before) + money} WHERE user_id={id}"
    start_dt = DT.strptime('01.01.2023', '%d.%m.%Y')
    end_dt = DT.strptime('01.02.2023', '%d.%m.%Y')
    income_date = str(get_random_date(start_dt, end_dt))[0:10]
    
    transaction_query = f"INSERT INTO \"transaction\""\
    "(timestamp, \"size\", \"type\", status, account_id, response_id) VALUES "\
    f"('{income_date}', {money}, 'incoming', 'done', {id}, null)" 
    cur.execute(account_query)
    cur.execute(transaction_query)

conn.commit()
cur.close()
conn.close()

In [5]:
# pay to system

import random
import psycopg2

conn = psycopg2.connect(user='postgres', password='1234', host='localhost', port='5432', database="youdo")
cur = conn.cursor()


for i in range(100):    
    user_id_query = 'SELECT user_id FROM "user" WHERE account::numeric::int >= 10'
    cur.execute(user_id_query)
    res = cur.fetchall()
    ids = []
    for i in res:
        ids.append(i[0])    
    id = random.choice(ids)
    cur.execute(f"SELECT account::numeric::int FROM \"user\" WHERE user_id={id}")
    before = cur.fetchall()[0][0]
    money = random.randint(10, before)
    start_dt = DT.strptime('01.02.2023', '%d.%m.%Y')
    end_dt = DT.strptime('01.04.2023', '%d.%m.%Y')
    outgo_date = str(get_random_date(start_dt, end_dt))[0:10]
    account_query = f"UPDATE \"user\" SET account={int(before) - money} WHERE user_id={id}"
    transaction_query = f"INSERT INTO \"transaction\""\
    "(timestamp, \"size\", \"type\", status, account_id, response_id) VALUES "\
    f"('{outgo_date}', {money}, 'outgoing', 'done', {id}, null)"
    cur.execute(account_query)
    cur.execute(transaction_query)

conn.commit()
cur.close()
conn.close()

In [16]:
# filling categories
import random
import psycopg2

conn = psycopg2.connect(user='postgres', password='1234', host='localhost', port='5432', database="youdo")
cur = conn.cursor()

cat_names = ["courier service", "repair and construction", "trucking",
             "cleaning and housekeeping assistance", "virtual assistant", 
             "computer help", "events and promotions", "design", 
             "software development", "photo, video and audio", 
             "installation and repair of equipment", "beauty and health",
             "digital equipment repair", "legal and accounting assistance",
             "contracts, documents, courts, taxes", "transport repair", "studying"]

for i in range(0, len(cat_names)):
    query = f"INSERT INTO category(\"name\")"\
            f" VALUES ('{cat_names[i]}');"
    cur.execute(query)
    
conn.commit()
cur.close()
conn.close()

In [50]:
# filling estimated cost for all categories

conn = psycopg2.connect(user='postgres', password='1234', host='localhost', port='5432', database="youdo")
cur = conn.cursor()

arr = range(1, 18)


# courier service
query = f"INSERT INTO estimated_cost(interval_begin, interval_end, response_cost, category_id) VALUES "\
f"(500, 600, 29, {arr[0]}), "\
f"(null, 500, 11, {arr[0]}), "\
f"(800, 1200, 60, {arr[0]}), "\
f"(1200, 1500, 99, {arr[0]}), "\
f"(1500, 2500, 141, {arr[0]}), "\
f"(2500, null, 199, {arr[0]}), "\
f"(600, 800, 42, {arr[0]}); "
cur.execute(query)

# repair and construction
query = f"INSERT INTO estimated_cost(interval_begin, interval_end, response_cost, category_id) VALUES "\
f"(null, 3000, 62, {arr[1]}), "\
f"(3000, 5000, 80, {arr[1]}), "\
f"(5000, 10000, 100, {arr[1]}), "\
f"(10000, 36000, 137, {arr[1]}), "\
f"(36000, null, 200, {arr[1]}); "
cur.execute(query)

# trucking
query = f"INSERT INTO estimated_cost(interval_begin, interval_end, response_cost, category_id) VALUES "\
f"(null, 2000, 30, {arr[2]}), "\
f"(2000, 5000, 50, {arr[2]}), "\
f"(5000, 8000, 80, {arr[2]}), "\
f"(8000, 15000, 100, {arr[2]}), "\
f"(15000, 24000, 131, {arr[2]}), "\
f"(24000, null, 199, {arr[2]});"
cur.execute(query)

# cleaning and housekeeping assistance
query = f"INSERT INTO estimated_cost(interval_begin, interval_end, response_cost, category_id) VALUES "\
f"(null, 1500, 38, {arr[3]}), "\
f"(1500, 3000, 53, {arr[3]}), "\
f"(3000, 5000, 76, {arr[3]}), "\
f"(5000, 8000, 91, {arr[3]}), "\
f"(8000, null, 109, {arr[3]});"
cur.execute(query)

# virtual assistant
query = f"INSERT INTO estimated_cost(interval_begin, interval_end, response_cost, category_id) VALUES "\
f"(null, 1000, 20, {arr[4]}), "\
f"(1000, 3000, 50, {arr[4]}), "\
f"(3000, 8000, 100, {arr[4]}), "\
f"(8000, 10000, 155, {arr[4]}), "\
f"(10000, null, 250, {arr[4]});"
cur.execute(query)

# computer help
query = f"INSERT INTO estimated_cost(interval_begin, interval_end, response_cost, category_id) VALUES "\
f"(null, 500, 30, {arr[5]}), "\
f"(500, 1500, 50, {arr[5]}), "\
f"(1500, 2500, 66, {arr[5]}), "\
f"(2500, 3500, 81, {arr[5]}), "\
f"(3500, null, 101, {arr[5]});"
cur.execute(query)

# events and promotions
query = f"INSERT INTO estimated_cost(interval_begin, interval_end, response_cost, category_id) VALUES "\
f"(null, 500, 22, {arr[6]}), "\
f"(500, 3500, 50, {arr[6]}), "\
f"(3500, 5000, 74, {arr[6]}), "\
f"(5000, 10000, 98, {arr[6]}), "\
f"(10000, null, 124, {arr[6]});"
cur.execute(query)

# design
query = f"INSERT INTO estimated_cost(interval_begin, interval_end, response_cost, category_id) VALUES "\
f"(null, 500, 26, {arr[7]}), "\
f"(500, 1500, 36, {arr[7]}), "\
f"(1500, 5000, 54, {arr[7]}), "\
f"(5000, 10000, 98, {arr[7]}), "\
f"(10000, 50000, 150, {arr[7]}), "\
f"(50000, null, 299, {arr[7]});"
cur.execute(query)

# software development
query = f"INSERT INTO estimated_cost(interval_begin, interval_end, response_cost, category_id) VALUES "\
f"(null, 1500, 57, {arr[8]}), "\
f"(1500, 5000, 80, {arr[8]}), "\
f"(5000, 10000, 90, {arr[8]}), "\
f"(10000, 25000, 122, {arr[8]}), "\
f"(25000, 50000, 150, {arr[8]}), "\
f"(50000, 100000, 180, {arr[8]}), "\
f"(100000, 150000, 201, {arr[8]}), "\
f"(150000, null, 299, {arr[8]});"
cur.execute(query)

# photo, video and audio
query = f"INSERT INTO estimated_cost(interval_begin, interval_end, response_cost, category_id) VALUES "\
f"(null, 500, 57, {arr[9]}), "\
f"(500, 1500, 69, {arr[9]}), "\
f"(1500, 3000, 90, {arr[9]}), "\
f"(3000, 6000, 126, {arr[9]}), "\
f"(6000, 15000, 185, {arr[9]}), "\
f"(15000, 25000, 256, {arr[9]}), "\
f"(25000, null, 298, {arr[9]});"
cur.execute(query)

# installation and repair of equipment
query = f"INSERT INTO estimated_cost(interval_begin, interval_end, response_cost, category_id) VALUES "\
f"(null, 1000, 53, {arr[10]}), "\
f"(1000, 2000, 70, {arr[10]}), "\
f"(2000, 5000, 90, {arr[10]}), "\
f"(5000, 10000, 134, {arr[10]}), "\
f"(10000, 15000, 187, {arr[10]}), "\
f"(15000, 30000, 256, {arr[10]}), "\
f"(30000, null, 289, {arr[10]});"
cur.execute(query)

# beauty and health
query = f"INSERT INTO estimated_cost(interval_begin, interval_end, response_cost, category_id) VALUES "\
f"(null, 500, 32, {arr[11]}), "\
f"(500, 1500, 50, {arr[11]}), "\
f"(1500, 3000, 70, {arr[11]}), "\
f"(3000, 5000, 121, {arr[11]}), "\
f"(5000, 15000, 240, {arr[11]}), "\
f"(15000, null, 300, {arr[11]});"
cur.execute(query)

# digital equipment repair
query = f"INSERT INTO estimated_cost(interval_begin, interval_end, response_cost, category_id) VALUES "\
f"(null, 1500, 34, {arr[12]}), "\
f"(1500, 3000, 58, {arr[12]}), "\
f"(3000, 10000, 70, {arr[12]}), "\
f"(10000, null, 123, {arr[12]});"
cur.execute(query)

# legal and accounting assistance
query = f"INSERT INTO estimated_cost(interval_begin, interval_end, response_cost, category_id) VALUES "\
f"(null, 500, 34, {arr[13]}), "\
f"(500, 1500, 58, {arr[13]}), "\
f"(1500, 5000, 100, {arr[13]}), "\
f"(5000, 7000, 165, {arr[13]}), "\
f"(7000, 15000, 241, {arr[13]}), "\
f"(15000, 40000, 357, {arr[13]}), "\
f"(40000, null, 456, {arr[13]});"
cur.execute(query)

# contracts, documents, courts, taxes
query = f"INSERT INTO estimated_cost(interval_begin, interval_end, response_cost, category_id) VALUES "\
f"(null, 1000, 50, {arr[14]}), "\
f"(1000, 3000, 123, {arr[14]}), "\
f"(3000, 10000, 254, {arr[14]}), "\
f"(10000, null, 368, {arr[14]});"
cur.execute(query)

# transport repair
query = f"INSERT INTO estimated_cost(interval_begin, interval_end, response_cost, category_id) VALUES "\
f"(null, 1500, 50, {arr[15]}), "\
f"(1500, 5000, 83, {arr[15]}), "\
f"(5000, 10000, 124, {arr[15]}), "\
f"(10000, 50000, 250, {arr[15]}), "\
f"(50000, null, 400, {arr[15]});"
cur.execute(query)

# studying
query = f"INSERT INTO estimated_cost(interval_begin, interval_end, response_cost, category_id) VALUES "\
f"(null, 1500, 100, {arr[16]}), "\
f"(1500, 5000, 200, {arr[16]}), "\
f"(5000, null, 500, {arr[16]});"
cur.execute(query)

conn.commit()
cur.close()
conn.close()

In [21]:
# fill review

import random
import psycopg2

conn = psycopg2.connect(user='postgres', password='1234', host='localhost', port='5432', database="youdo")
cur = conn.cursor()

# get ids
user_ids_query = "SELECT user_id FROM \"user\""
cur.execute(user_ids_query)
res = cur.fetchall()
ids = []
for i in res:
    ids.append(i[0])
    
for i in range(100):
    id1 = random.choice(ids)
    id2 = random.choice(ids)
    while (id1 == id2):
        id2 = random.choice(ids)
    NAME_Good1 = ["Good", "Excellent", "Cute", "Grand", "Ideal", "Important", 
                  "Quick"]
    NAME_Bad1 = ["Awfull", "Hard", "Useless", "So expensive", "Really bad", 
                 "Bad", "The worst", "Failed", "Important", "Long"]
    NAME_2 = ["work", "task", "experience", "job", "diagnosis", "deal"]
    START_Bad = ["So", "Unfortunately", "I am angry because", "To be honest,"]
    START_Good = ["So", "Fortunately", "I am happy because", "Great that"]
    END = ["pay for this again", "recommend this performer", 
           "use this service again", "choose this performer again"]
    GRADE = ["1", "2", "3", "4", "5"]
    flag1 = random.randint(0, 100)
    if (flag1 > 20):
        grade1 = random.choice(GRADE[2:5])
        grade2 = random.choice(GRADE[3:5])
        grade3 = random.choice(GRADE[2:5])
        punc = random.choice([".", ")", ")!", "))))" "!"])
        description = f"{random.choice(START_Good)} it was "\
        f"{random.choice(NAME_Good1).lower()} {random.choice(NAME_2)} "\
        f"and I will {random.choice(END)}{punc}"
    else:
        grade1 = random.choice(GRADE[0:2])
        grade2 = random.choice(GRADE[0:4])
        grade3 = random.choice(GRADE[0:4])
        punc = random.choice(["...", ".", "(", "(((" "!", "!!!"])
        description = f"{random.choice(START_Bad)} it was "\
        f"{random.choice(NAME_Bad1).lower()} {random.choice(NAME_2)} "\
        f"and I will not {random.choice(END)}{punc}"
    review_query = f"INSERT INTO review"\
    f"(description, quality_grade, politeness_grade, cost_grade,"\
    " object_id, author_id) VALUES "\
    f"(\'{description}\', '{grade1}', '{grade2}', '{grade3}', "\
    f" {id1}, {id2})" 
    cur.execute(review_query)

conn.commit()
cur.close()
conn.close()

In [31]:
# fill task!

import random
import psycopg2
import pandas

conn = psycopg2.connect(user='postgres', password='1234', host='localhost', port='5432', database="youdo")
cur = conn.cursor()

# get user ids
user_ids_query = "SELECT user_id FROM \"user\""
cur.execute(user_ids_query)
res = cur.fetchall()
user_ids = []
for i in res:
    user_ids.append(i[0])
    
# get category_id and estimated_cost_id
query = "SELECT cost_id, category_id FROM estimated_cost"
cur.execute(query)
res = cur.fetchall()
pair_ids = []
for i in res:
    pair_ids.append(i)

for i in range(10):
    # choosing task status, author and performer ids
    if(random.randint(0, 100) > 99):
        task_status = "published"
    else:
        if(random.randint(1, 100) > 99):
            task_status = "in process"
        else:
            task_status = "done"
    id1 = random.choice(user_ids)
    # choosing category and cost
    estimated_cost_id, category_id = random.choice(pair_ids)
    # choose start and end time
    start_time = str(random.choice(pandas.date_range('2023-02-01', '2023-03-31')))[:-9]
    end_time = str(random.choice(pandas.date_range(start_time, '2023-04-02')))[:-9]
    #generate name
    NAME_1 = ["Tricky", "Unusual", "Good", "Hard", "Grand", "Careful", "Quick", "Important", "Easy", "Small", "Big"]
    NAME_2 = ["work", "task", "job", "case"]
    name = random.choice([f"{random.choice(NAME_1)} {random.choice(NAME_2)}", 
                          f"Help with {random.choice(NAME_1).lower()}"\
                          f" {random.choice(NAME_2)}"])
    description = f"I need {random.choice(['help to', 'a professional to','a person to', 'to'])} "\
                f"do some {random.choice(NAME_1).lower()} {random.choice(NAME_2)} "\
                f"and also I {random.choice(['need materials', 'have busy schedule','need to get some advices','have an angry dog','live far from metro', 'do not have much time'])}"
    task_query = f"INSERT INTO task(\"name\", description, begin_time, "\
    f"end_time, status, author_id, estimated_cost_id) VALUES "\
    f"('{name}', \'{description}\', '{start_time}', '{end_time}', "\
    f" '{task_status}', {id1}, {estimated_cost_id})" 
    task_query += ' RETURNING task_id'
    cur.execute(task_query)
    task_id = cur.fetchall()[0][0]
    
    resp_cnt = random.choice(range(15))
    
    if(resp_cnt == 0): 
        continue
    
    for j in range(resp_cnt):
        author_id = id1
        while (author_id == id1):
            author_id = random.choice(user_ids)
        description = f'{random.choice(["I will", "Let us", "I am glad to", "I am ready to"])}'\
                    f' {random.choice(["do", "solve", "deal with"])} this'\
                    f' {random.choice(NAME_1).lower()} {random.choice(NAME_2)}.'
        query = f"SELECT interval_begin::numeric::int, interval_end::numeric::int, response_cost::numeric::int FROM estimated_cost WHERE cost_id = {estimated_cost_id}"
        cur.execute(query)
        f = cur.fetchone()
        
        cost_begin = f[0]
        cost_end = f[1]
        if(type(cost_begin) != type(1)):
            cost_begin = 0
        if(type(cost_end) != type(1)):
            cost_end = 3*cost_begin
        cost = random.choice(range(cost_begin, cost_end))
        
        response_cost = f[2]
        query = f"SELECT account::numeric::int FROM \"user\" WHERE user_id = {author_id}"
        cur.execute(query)
        current_account = cur.fetchone()[0]
        if(current_account < response_cost): 
            continue
        
        response_query = f'INSERT INTO response(description, \"cost\", author_id, task_id) VALUES'\
        f"(\'{description}\', {cost}, {author_id}, {task_id}) RETURNING response_id"
        cur.execute(response_query)
        response_id = cur.fetchone()[0] 
        
        account_query = f"UPDATE \"user\" SET account={int(current_account) - response_cost} WHERE user_id={author_id}"
        cur.execute(account_query)
        start_dt = DT.strptime('01.04.2023', '%d.%m.%Y')
        end_dt = DT.strptime('15.04.2023', '%d.%m.%Y')
        outgo_date = str(get_random_date(start_dt, end_dt))[0:10]
        transaction_query = f"INSERT INTO \"transaction\""\
        "(timestamp, \"size\", \"type\", status, account_id, response_id) VALUES "\
        f"('{outgo_date}', {response_cost}, 'outgoing', 'done', {author_id}, {response_id})"
        cur.execute(transaction_query)
        
    if(task_status == "published"): 
        continue
    
    cur.execute(f"SELECT response_id FROM response WHERE task_id = {task_id}")
    res = cur.fetchall()
    resp_ids = []
    for i in res:
        resp_ids.append(i[0])
    if(len(resp_ids) == 0):
        continue
    assigned_response = random.choice(resp_ids)
    cur.execute(f"UPDATE task SET assigned_response_id = {assigned_response} WHERE task_id = {task_id}")
        
    if(task_status == "in process"): 
        continue
    
    query = f"SELECT \"cost\"::numeric::int, author_id FROM response WHERE response_id = {assigned_response}"
    cur.execute(query)
    res = cur.fetchone()
    cost = res[0]
    worker_id = res[1]
    
    cur.execute(f"SELECT account::numeric::int FROM \"user\" WHERE user_id={id}")
    customer_before = cur.fetchone()[0]
    if(cost > customer_before):
        cur.execute(f"UPDATE task SET status = 'in process' WHERE task_id = {task_id}")
        continue
    cur.execute(f"SELECT account::numeric::int FROM \"user\" WHERE user_id={worker_id}")
    worker_before = cur.fetchone()[0]
    start_dt = DT.strptime('01.04.2023', '%d.%m.%Y')
    end_dt = DT.strptime('01.05.2023', '%d.%m.%Y')
    outgo_date = str(get_random_date(start_dt, end_dt))[0:10]
    account_query1 = f"UPDATE \"user\" SET account={int(customer_before) - cost} WHERE user_id={id}"
    account_query2 = f"UPDATE \"user\" SET account={int(worker_before) + cost} WHERE user_id={worker_id}"
    transaction_query = f"INSERT INTO \"transaction\""\
    "(timestamp, \"size\", \"type\", status, account_id, response_id) VALUES "\
    f"('{outgo_date}', {cost}, 'outgoing', 'done', {id}, {assigned_response}), "\
    f"('{outgo_date}', {cost}, 'incoming', 'done', {worker_id}, {assigned_response})"
    cur.execute(account_query1)
    cur.execute(account_query2)
    cur.execute(transaction_query)    
    
conn.commit()
cur.close()
conn.close()